In [2]:
import os
import cv2
import PIL
from PIL import Image, ImageChops
import threading
import queue

In [12]:
print('указать путь', end = '')
directory_in_str = input()

directory = os.fsencode(directory_in_str)
imgs = os.listdir(directory_in_str)
dublicates = {}
try:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        last_name = directory_in_str
        last_name += '\\' + filename
        f_info = os.stat(last_name)
        if f_info.st_size in dublicates:
            image_1 = Image.open(last_name)
            image_2 = Image.open(dublicates[f_info.st_size])
            result = ImageChops.difference(image_1, image_2)
            result = result.getbbox()
            couter = 0
            if result == None:
                couter += 1
                print(f'Найден дубликат: [{last_name} и {dublicates[f_info.st_size]}]')
            else:
                dublicates[f_info.st_size] = last_name
except:
    print('Ошибка поиска директории!')

print(f'Программа завершила работу, файлов обработано .')

указать путь

 E:/tehzor/datasets/axes_op/jpgs/


Программа завершила работу, файлов обработано .


In [2]:
from PIL import ImageChops

image_1=Image.open('E:/tehzor/datasets/axes_op/jpgs/5a3ff89583f47602aa303c70.jpg')
image_2=Image.open('E:/tehzor/datasets/axes_op/jpgs/5a3ffbbdbe6da402a9547a33.jpg')

result=ImageChops.difference(image_1, image_2)
result.show()

#Вычисляет ограничивающую рамку ненулевых областей на изображении.
print(result.getbbox()) 

# result.getbbox() в данном случае вернет (0, 0, 888, 666)
result.save('result.jpg')

None


In [3]:
def difference_images(img1, img2):
    image_1 = Image.open(img1)
    image_2 = Image.open(img2)
    
    size = [400,300]        #размер в пикселях
    image_1.thumbnail(size) #уменьшаем первое изображение
    image_2.thumbnail(size) #уменьшаем второе изображение

    #сравниваем уменьшенные изображения
    result=ImageChops.difference(image_1, image_2).getbbox() 
    if result==None:
        print(img1,img2,'matches')
    return

In [7]:
path='E:/tehzor/datasets/axes_op/jpgs/' #Путь к папке где лежат файлы для сравнения
#imgs=os.listdir(path)


        

class diff_image(threading.Thread): #класс по сравнению картинок.
    """Потоковый обработчик"""
    def __init__(self, queue):
        """Инициализация потока"""
        threading.Thread.__init__(self)
        self.queue = queue
    def run(self):
        """Запуск потока"""
        while True:
            # Получаем пару путей из очереди
            files = self.queue.get()
            # Делим и сравниваем
            self.difference_images(files.split(':')[0],files.split(':')[1])        
            # Отправляем сигнал о том, что задача завершена
            self.queue.task_done()
            
    def difference_images(self, img1, img2):
        image_1 = Image.open(img1)
        image_2 = Image.open(img2)

        size = [400,300] #размер в пикселях
        image_1.thumbnail(size) #уменьшаем первое изображение
        image_2.thumbnail(size) #уменьшаем второе изображение

        result=ImageChops.difference(image_1, image_2).getbbox()
        if result==None:
            print(img1,img2,'matches')
        return 
        
def main(path):
    imgs=os.listdir(path) #Получаем список картинок
    q = queue.Queue()
    
    # Запускаем поток и очередь
    for i in range(4): # 4 - кол-во одновременных потоков
        t = diff_image(q)
        t.setDaemon(True)
        t.start()    
       
    # Даем очереди нужные пары файлов для проверки
    check_file=0
    current_file=0

    while check_file<len(imgs):
        if current_file==check_file:
            current_file+=1
            continue
        q.put(path+imgs[current_file]+':'+path+imgs[check_file]) 
        current_file+=1
        if current_file==len(imgs):
            check_file+=1
            current_file=check_file   

    # Ждем завершения работы очереди
    q.join()


if __name__ == "__main__":
    path='E:/tehzor/datasets/axes_op/jpgs/'
    main(path)

Exception in thread Thread-17:
Traceback (most recent call last):
  File "C:\Users\olga\miniconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\olga\AppData\Local\Temp\ipykernel_19564\141478493.py", line 19, in run
    self.difference_images(files.split(':')[0],files.split(':')[1])
  File "C:\Users\olga\AppData\Local\Temp\ipykernel_19564\141478493.py", line 24, in difference_images
    image_1 = Image.open(img1)
  File "C:\Users\olga\miniconda3\lib\site-packages\PIL\Image.py", line 3068, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'E'

Exception in thread Thread-18:
Traceback (most recent call last):
  File "C:\Users\olga\miniconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\olga\AppData\Local\Temp\ipykernel_19564\141478493.py", line 19, in run
    self.difference_images(files.split(':')[0],files.split(':')[1])
  File "C:\Users\olga\AppData\Local\Temp\ipy

IndexError: list index out of range